In [2]:
!pip install qwen_vl_utils
!pip install transformers
!pip install datasets
!pip install -U bitsandbytes

import torch
from transformers import (
    Qwen2VLForConditionalGeneration, AutoProcessor,
    Trainer, TrainingArguments,
    BitsAndBytesConfig, Qwen2VLProcessor
)
from qwen_vl_utils import process_vision_info
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
import torch
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

compute = True

# Check device compatibility
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
print('device = ', device)

model_name = "Qwen/Qwen2-VL-2B-Instruct"

img_folder_path = "/content/drive/MyDrive/MATH-V-main"

dataset_name = "MathLLMs/MathVision"

file_name = f"data/output/{dataset_name.split('/')[-1]}_{model_name.split('/')[-1]}.csv"


# Load the dataset
data_list = load_dataset(dataset_name, split='test')


# Split data into training and validation sets
train_size = int(0.7 * len(data_list))
val_size = int(0.1 * len(data_list))
test_size = len(data_list) - train_size - val_size
data_list = data_list.shuffle(seed=42)  # Shuffle the dataset for randomness

train_data = data_list.select(range(train_size))
val_data = data_list.select(range(train_size, train_size + val_size))
test_data = data_list.select(range(train_size+val_size, len(data_list)))


Mounted at /content/drive
device =  cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

(…)-00000-of-00001-3532b8d3f1b4047a.parquet:   0%|          | 0.00/57.0M [00:00<?, ?B/s]

(…)-00000-of-00001-f8ff70fcb2f29b1d.parquet:   0%|          | 0.00/6.99M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/3040 [00:00<?, ? examples/s]

Generating testmini split:   0%|          | 0/304 [00:00<?, ? examples/s]

In [6]:
def generate_text_from_sample(model, processor, sample, max_new_tokens=1024):
    image_path = f"{img_folder_path}/{input['image']}"
    prompt = (input["question"])

    # Prepare input
    messages = [
        {"role": "user", "content": [{"type": "image", "image": image_path}, {"type": "text", "text": prompt}]}
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )

    # Move inputs to the device
    inputs = inputs.to(device)
    model = model.to(device)

    # Perform inference
    generated_ids = model.generate(**inputs, max_new_tokens=256)

    # Trim the generated ids to remove the input ids
    trimmed_generated_ids = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(model_inputs.input_ids, generated_ids)]

    output_text = processor.batch_decode(
        trimmed_generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]
    return output_text


In [5]:

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    # quantization_config=bnb_config
)
processor = Qwen2VLProcessor.from_pretrained(model_name)

from peft import LoraConfig, get_peft_model

# Configure LoRA
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

# Apply PEFT model adaptation
peft_model = get_peft_model(model, peft_config)

# Print trainable parameters
peft_model.print_trainable_parameters()

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

trainable params: 1,089,536 || all params: 2,210,075,136 || trainable%: 0.0493


In [ ]:
# test

# Load the model
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float32, #if device == "cpu" else torch.bfloat16,
    device_map=None
)

# Initialize processor
min_pixels = 256 * 28 * 28
max_pixels = 1280 * 28 * 28
processor = AutoProcessor.from_pretrained(model_name, min_pixels=min_pixels, max_pixels=max_pixels)


results = []

for i, input in enumerate(tqdm(test_data)):
    output_text = generate_text_from_sample(model, processor, input)

    # Store results
    results.append(input | {
        "generated_text": output_text
    })
    # if i > 4:
    #   break


pd.DataFrame(results).to_csv(f"{img_folder_path}/test_generated_answer.csv")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
from transformers import (
    Qwen2VLForConditionalGeneration,
    AutoTokenizer,
    AutoImageProcessor
)
from PIL import Image
import os
from typing import Dict, List

class MathVQADataset(Dataset):
    def __init__(self, data: List[Dict], tokenizer, image_processor, max_length=512, image_size=(448, 448)):  # Updated image size
        self.data = data
        self.tokenizer = tokenizer
        self.image_processor = image_processor
        self.max_length = max_length
        self.image_size = image_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Load and process image
        image = Image.open(f"{img_folder_path}/{item['image']}").convert('RGB')

        # Process image with all required parameters
        image_features = self.image_processor(
            image,
            return_tensors="pt",
            do_resize=True,
            size={"height": self.image_size[0], "width": self.image_size[1]},
            do_normalize=True
        )

        # Create prompt with system message
        prompt = f"<|im_start|>system\nYou are a helpful assistant that answers math questions based on images.<|im_end|>\n<|im_start|>user\nQuestion: {item['question']}<|im_end|>\n<|im_start|>assistant\nLet me solve this step by step.\nAnswer:"
        target = f"{item['answer']}<|im_end|>"

        # Tokenize input and target
        inputs = self.tokenizer(
            prompt,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        targets = self.tokenizer(
            target,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': targets['input_ids'].squeeze(),
            'pixel_values': image_features['pixel_values'].squeeze(),
        }

def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    pixel_values = torch.stack([item['pixel_values'] for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels,
        'pixel_values': pixel_values
    }

def train_model(train_data: List[Dict], validation_data: List[Dict], model_save_name: str):
    # Initialize model, tokenizer, and image processor with trust_remote_code=True
    model_name = "Qwen/Qwen2-VL-2B-Instruct"
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype="auto", device_map="auto"  # Enable automatic device mapping
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    image_processor = AutoImageProcessor.from_pretrained(model_name, trust_remote_code=True)


    # # Preparation for inference
    # text = processor.apply_chat_template(
    #     messages, tokenize=False, add_generation_prompt=True
    # )
    # image_inputs, video_inputs = process_vision_info(messages)
    # inputs = processor(
    #     text=[text],
    #     images=image_inputs,
    #     videos=video_inputs,
    #     padding=True,
    #     return_tensors="pt",
    # )
    # inputs = inputs.to("cuda")

    # # Inference: Generation of the output
    # generated_ids = model.generate(**inputs, max_new_tokens=128)
    # generated_ids_trimmed = [
    #     out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    # ]
    # output_text = processor.batch_decode(
    #     generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    # )

    # Set output directory
    output_dir = f"./fine_tuned_{model_save_name}"
    os.makedirs(output_dir, exist_ok=True)

    # Create datasets
    train_dataset = MathVQADataset(train_data, tokenizer, image_processor)
    val_dataset = MathVQADataset(validation_data, tokenizer, image_processor)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=2,  # Reduced batch size
        per_device_eval_batch_size=2,   # Reduced batch size
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=os.path.join(output_dir, 'logs'),
        logging_steps=100,
        evaluation_strategy="steps",
        eval_steps=500,
        save_steps=1000,
        learning_rate=2e-5,
        fp16=True,
        gradient_accumulation_steps=8,  # Increased gradient accumulation
        save_total_limit=3,
        remove_unused_columns=False,
        dataloader_num_workers=2
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=collate_fn,
    )

    # Start training
    trainer.train()

    # Save the final model
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)
    image_processor.save_pretrained(output_dir)

# Usage
model_save_name = "math_vqa_model"
train_model(train_data, val_data, model_save_name)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_prompt(example, prompt_type ='full', context=''):
    question = example['question']
    options = ''
    if len(example['options']) > 0:
        assert len(example['options']) == 5, example
        if ''.join(example['options']) != 'ABCDE':
            options = f"(A) {example['options'][0]}\n(B) {example['options'][1]}\n(C) {example['options'][2]}\n(D) {example['options'][3]}\n(E) {example['options'][4]}\n"

    # input = 'Please solve the problem step by step and put your answer in one "\\boxed{}". If it is a multiple choice question, only one letter is allowed in the "\\boxed{}".\n'+f"{question}\n{options}"
    if prompt_type == 'simple':
      prompt = f"{question}\n{options}\nAnswer the question using a single word or phrase."
    else:
      prompt = f"""{context}
        Question: {question}\n{options}
        """ + 'Please solve the problem by elaborately, providing step-by-step reasoning for the solution and put your answer in one "\\boxed{}. Also, provide the bounding box coordinate of the region that can help you answer the question better.If it is a multiple choice question, only one letter is allowed in the \\boxed{}.\n'
    return prompt

In [ ]:
import re
remove_list = ['the answer', 'is', ':', 'boxed', '$']

results = pd.DataFrame(results)
def final_answer(text: str):
    text = text.lower()
    if 'answer is' not in text:
        return np.nan
    # Create a regex pattern to match all words in the remove_list
    pattern = r'(' + r'|'.join(re.escape(word) for word in remove_list) + r'|\s)'
    # Use re.sub to replace matched patterns with an empty string
    result = re.sub(pattern, '', text.split('answer is')[-1])
    result = re.sub(r'\\\{(\\frac\{[^{}]+\}\{[^{}]+\})\}', r'\1', result)
    return re.sub(r'\\\\', r'\\', result)
# Example metric: String matching (very basic)
results['prediction'] = results['generated_text'].apply(final_answer)
results['exact_match'] = results['prediction'] == results['answer']
display(results)
pd.DataFrame(results).to_csv(f"{img_folder_path}/test_generated_answer.csv")

accuracy = sum(results['exact_match']) / len(results)
print(f"Exact Match Accuracy: {accuracy:.4f}")

# Optionally save results
# import json
# with open("evaluation_results.json", "w") as f:
#     json.dump(results, f, indent=4)

,id,question,options,image,decoded_image,answer,solution,level,subject,prompt,generated_text,prediction,exact_match
0,2941,A sphere is inscribed in a cone with height 4 ...,[],images/2941.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,\frac{3}{8},We start by finding the radius of the inscribe...,2,solid geometry,A sphere is inscribed in a cone with height 4 ...,system\nYou are a helpful assistant.\nuser\nA ...,\frac{3}{1}.,False
1,2942,The length of the diameter of this spherical b...,[],images/2942.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,27000-4500\pi,The box has volume $30^3=27000$ cubic cm.\n\nT...,1,solid geometry,The length of the diameter of this spherical b...,system\nYou are a helpful assistant.\nuser\nTh...,NaN,False
2,2943,"In the circle below, $\overline{AB} \| \overli...",[],images/2943.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,8\pi,"By symmetry, $\widehat{BD}=\widehat{CA}=100^\c...",4,metric geometry - length,"In the circle below, $\overline{AB} \| \overli...",system\nYou are a helpful assistant.\nuser\nIn...,NaN,False
3,2944,"In right triangle $ABC$, $\angle B = 90^\circ$...",[],images/2944.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,2\sqrt{3},"Let $\overline{DE}$ have length $x$, so $\over...",4,metric geometry - length,"In right triangle $ABC$, $\angle B = 90^\circ$...",system\nYou are a helpful assistant.\nuser\nIn...,NaN,False
4,2945,The area of square $ABCD$ is 100 square centim...,[],images/2945.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,68,"Since $AE = 2$, $EB = 8$, but since $EFGH$ is ...",1,metric geometry - area,The area of square $ABCD$ is 100 square centim...,system\nYou are a helpful assistant.\nuser\nTh...,100,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3036,The two externally tangent circles each have a...,[],images/3036.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,8-2\pi,Each diameter of a circle is 2 units. The rect...,1,metric geometry - area,The two externally tangent circles each have a...,system\nYou are a helpful assistant.\nuser\nTh...,NaN,False
96,3037,The area of $\triangle ABC$ is 6 square centim...,[],images/3037.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,54,"Since $AB \parallel DE,$ we know that $\angle ...",4,metric geometry - area,The area of $\triangle ABC$ is 6 square centim...,system\nYou are a helpful assistant.\nuser\nTh...,NaN,False
97,3038,"In the diagram, $K$, $O$ and $M$ are the cente...",[],images/3038.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1250\pi,We know that $OA$ and $OB$ are each radii of t...,4,metric geometry - area,"In the diagram, $K$, $O$ and $M$ are the cente...",system\nYou are a helpful assistant.\nuser\nIn...,512\pi,False
98,3039,The volume of the cylinder shown is $45\pi$ cu...,[],images/3039.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,5,The volume of the cylinder is $bh=\pi r^2h$. T...,1,solid geometry,The volume of the cylinder shown is $45\pi$ cu...,system\nYou are a helpful assistant.\nuser\nTh...,NaN,False


Exact Match Accuracy: 0.0200
